<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/05-Time_Series/A-Introduction_to_Time_Series_using_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Time Series and Forecasting

*Based on the book [Introduction to Time Series and Forecasting](https://link.springer.com/book/10.1007/978-3-319-29854-2) by Brockwell and Davis.*



In [ ]:
#@title Setup

!sudo pip install -U -q PyMySQL sqlalchemy sql_magic 

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

matplotlib.style.use(['seaborn-talk', 'seaborn-ticks', 'seaborn-whitegrid'])
%config InlineBackend.figure_format = 'retina'

## What is a Time Series?

A time series is a set of observations $x_t$, each one being recorded at a specific time $t$. In our sessions, we focus on **_discrete_** time series, where observations are recorded at fixed time intervals (e.g., once an hour, or every 30 seconds, or every 7 days). 

We only consider **_regular_** time series, where we the time between observations is constant (i.e., we do not consider account deposits or withdrawals from an ATM that happen at various times; these are examples of an irregular time series).

## Examples of Time Series



### Australian red wine "sales", (thousands of litres) monthly, Jan 80 - Oct 91

The file [`australian-wine-sales.txt`](https://storage.googleapis.com/datasets_nyu/australian-wine-sales.txt) contains the monthly sales of Australian red wines in for the period Jan-1980 to Oct-1991. Let's take a peak at the data file: We will use Pandas and the `pd.read_csv` function to read the text file into a dataframe. 

In [ ]:
url = "https://storage.googleapis.com/datasets_nyu/australian-wine-sales.txt"
df = pd.read_csv(url, sep='\t')

##### What is this code?
# The `read_csv` command can read directly from a URL, so we pass directly 
# the URL of the dataset as a parameter. Also, since the file uses the 
# tab character to separate the columns, we pass the `sep='\t'` option to 
# the `read_csv` command, indicating that the separator is the "tab"
# (i.e. `\t` ) character.


In [ ]:
df.head(10)

We can try to plot the time series:

In [ ]:
df.plot()

The plot would look better if we had the x-axis to be a date, instead of a number. By default, Pandas uses the "index" of the dataframe as the x-axis.



Let's check the data types that Pandas inferred:

In [ ]:
df.dtypes

In this case pandas figured out that `Sales` is a number, but not that `Date` is a date. The following two commands convert the two columns into a date and a numeric data type, respectively. (Technically, we could skip the conversion for `Sales` but I want to show how we convert data types.)

We will set  the `Date` column to be the index of the dataframe, so that we can plot the sales with dates as the x-axis.

In [ ]:
df["Date"] = pd.to_datetime(df["Date"])
df["Sales"] = pd.to_numeric(df["Sales"])

In [ ]:
df = df.set_index("Date")
df

In [ ]:
df.plot()

It appears from the graph that the sales have an upward trend and a seasonal pattern with a peak in July and a trough in January.

### Typical questions in Time Series Analysis

* What is the overall trend?
* Is the current month above or below expectations?
* Do we observe any anomalies in our data?
* What should we expect for next month? Next year?

To answer such questions, we typically start by considering a few **models** of time series, and see how well such models capture the behavior of the data that we have. 

For example, it is important to recognize the presence of **seasonal components** and to remove them so as not to confuse them with long-term trends. This process is known as **seasonal adjustment**.



### More Time Series

#### NYC Accidents

Taking a look at the aggregate data for NYC accidents, we can start asking questions like:

* Does the trend look good?
* When do we observe spikes in accidents on a daily and weekly basis, so that we can deploy resources accordingly? (We can also do that on a geographical basis, but we will examine that separately)
* Which dates or times are unusually high or low?

In [ ]:
# Connect to the database
conn_string = 'mysql+pymysql://{user}:{password}@{host}/?charset=utf8mb4'.format(
    host = 'db.ipeirotis.org', 
    user = 'student',
    password = 'dwdstudent2015', 
    encoding = 'utf8mb4')
mysql_conn = create_engine(conn_string).connect()

# Get the number of accidents per hour
sql = '''
  SELECT date_format(DATE_TIME,'%%Y-%%m-%%d %%H:00') AS acc_date, COUNT(*) AS accidents 
  FROM collisions.collisions 
  GROUP BY date_format(DATE_TIME,'%%Y-%%m-%%d %%H:00')
  ORDER BY date_format(DATE_TIME,'%%Y-%%m-%%d %%H:00')
'''

# Read the results in Pandas
acc_hourly = pd.read_sql(sql, con=mysql_conn)

# Convert the acc_date column to datetime
acc_hourly['acc_date'] = pd.to_datetime(acc_hourly['acc_date'])

acc_hourly.plot(
    kind='line',
    x='acc_date',
    y='accidents'
)

We can change the granularity of the time series by calculating aggregate statistics:

In [ ]:
#### NYC Accidents Daily/Weekly/Monthly

acc_aggregated = acc_hourly.copy()
acc_aggregated = acc_aggregated.set_index('acc_date')

# Modify the aggregation here to get values for daily, weekly, monthly, etc
acc_aggregated = acc_aggregated.resample('1W').sum()

# Plot the aggregated data; by default, x-axis is the index
acc_aggregated.plot(
    kind='line',
    y='accidents'
)

#### The monthly accidental deaths data, 1973–1978

For more details on the data manipulation aspect of the commands below, see the [Reading Data in Pandas](https://github.com/ipeirotis/dealing_with_data/blob/master/03-Pandas/B1-Pandas_Reading_Data.ipynb) notebook, in the section "_Loading a 'Fixed Width' file_".

In [ ]:
deaths = pd.read_fwf("https://storage.googleapis.com/datasets_nyu/acc-deaths.txt")
deaths = pd.melt(deaths, id_vars=['Year'], 
        value_vars=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        var_name='Month', value_name='Deaths')
deaths["Date"] = deaths["Month"] + "-" + deaths["Year"].astype(str)
deaths["Date"] = pd.to_datetime(deaths["Date"], format='%b-%Y')
deaths = deaths.drop(["Month","Year"], axis='columns')
deaths = deaths.set_index(keys="Date")
deaths = deaths.sort_index()
deaths.plot()

#### US Population

In [ ]:
# We use the thousands=',' option to properly convert the population numbers to integers
population = pd.read_csv("https://storage.googleapis.com/datasets_nyu/us-population.txt", sep=' ', thousands=',')
population["Year"] = pd.to_numeric(population["Year"])
population["US_Population"] = pd.to_numeric(population["US_Population"])

population.plot(
    kind = 'line',
    x = 'Year',
    y = 'US_Population',
    marker = 'o'
)

#### Stock prices

If you are interested in downloading stock information, [this article contains a good discussion](https://towardsdatascience.com/a-comprehensive-guide-to-downloading-stock-prices-in-python-2cd93ff821d4).

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf

stock_df = yf.download(tickers = ['GOOG','AAPL','MSFT'],
                       interval = '1d', # download daily prices
                       start='2005-01-01', # fetch prices after 2004
                       auto_adjust = True, # adjust for splits etc
                       progress = False # do not show a progress bar
                       )
stock_df

In [ ]:
stock_df.plot(
    y = 'Close',
    title="Daily Closing price",
    logy=True
)

In [ ]:
# Variation of the plot, where we normalize all stocks
# by dividing the stock with the price on the first day
# of 2005.
normalized_stock_df = stock_df['Close'].copy()
first_date = normalized_stock_df.index[0]
first_entry = normalized_stock_df.loc[first_date]
normalized_stock_df = normalized_stock_df / first_entry

normalized_stock_df.plot(
    title="Normalized Price (y=1 is at Jan 1 2005)",
    logy=True
)

In [ ]:
normalized_stock_df.index[0]

## Analyzing Time Series: Autocorrelation

A commonly analyzed property of a time series is the **autocorrelation** of the sequential observations. Simply stated, a high degree of autocorrelation means that if we know the value at time $t$, we can predict well the value at $t+1$.

Using the `autocorr` function, we estimate the autocorrelation of our `Sales` time series:

In [ ]:
df["Sales"].autocorr()

The concept of autocorrelation can extend to longer time periods, and not just to $t$ and $t+1$. We can extract autocorrelation for various **lag** values.

In [ ]:
df["Sales"].autocorr(lag=1) # same as simply df["Sales"].autocorr()

In [ ]:
# Correlation between t and t+2
# ie sales now and 2 months later
df["Sales"].autocorr(lag=2)

In [ ]:
# Correlation between t and t+3
# ie sales now and 3 months later
df["Sales"].autocorr(lag=3)

### Lag plots and  autocorrelation plots

Pandas provides two types of plots that can be used for the analysis of time series: the `lag_plot` and the `autocorrelation_plot`. We can also use the seasonal decomposition functionality of `statsmodels` to separate the time series into a trend, seasonal component, and residual noise. We will go quickly over these for now, mainly for demo purposes. Proper treatment of these topics require deeper analysis.



#### Lag plot

By default, the lag plot shows the value of the series at time $t$ vs. its value at time $t+1$. If there is no dependency (i.e., the time series is noise) then the lag plot is a scatterplot without any sign of correlation. If we can see a pattern and a correlation, then the series exhibits autocorrelation. For example, below we can see that there is a rather strong correlation of the two variables, indicating that the sales in time $t+1$ is similar to the sales at time $t$.

In [ ]:
pd.plotting.lag_plot(df["Sales"], lag = 1, c='b')

In [ ]:
# The plot above shows the autocorrelation of the t and t+1
# is around 0.73
df["Sales"].autocorr(lag=1)

Here is the lag plot, where we plot $t$ and $t+12$. Notice that we have a higher correlation (less spread out points)

In [ ]:
pd.plotting.lag_plot(df["Sales"], lag = 12, c='r')

In [ ]:
df["Sales"].autocorr(lag=12)

Let's plot the two of them together.

In [ ]:
pd.plotting.lag_plot(df["Sales"], lag = 1, c='b')
pd.plotting.lag_plot(df["Sales"], lag = 12, c='r')

#### Autocorrelation Plot

In a more general setting, we want to also see if the value of the series at time $t$ is predictive of the value at time $t+n$. Such dependency would indicate that there is *autocorrelation* in the series. The autocorrelation plot shows the correlation value for various values of $n$.

In [ ]:
pd.plotting.autocorrelation_plot(df["Sales"])

The plot above, with the oscillating autocorrelation values indicate that there is a **seasonality** component in the time series. (As we see that the correlation in 12-month increments to go up and then down.) 

Let's see next how we can extract the seasonal component.

## Trend and Seasonal Decomposition



In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# We decompose assumming a 12-month periodicity. 
# We can also specify a multiplicative instead of an additive model
# The additive model is Y[t] = T[t] + S[t] + e[t]
# The multiplicative model is Y[t] = T[t] * S[t] * e[t]
decomposition = seasonal_decompose(df['Sales'], model='multiplicative', freq=12, extrapolate_trend='freq')  
fig = plt.figure()
fig = decomposition.plot()

#### Accessing indinvidual components of the decomposition

Once we have the decomposed time series model, we can also access the different components.

For example, we can get the trend of the time series, after removing the seasonality component:

In [ ]:
# The outcome is a pandas Series, which is effectively the same as a single column of dataframe
decomposition.trend

In [ ]:
decomposition.trend.plot()

## Exercise

* Below we fetch the daily number of vehicular accidents in NYC.
* Examine the autocorrelation structure of the accidents.
* Perform a decomposition of the time series into a trend, seasonal, and residual component.
* Try out both the additive and the multiplicative approach for the decomposition. Try to interpret and understand the difference in the reported seasonal component.
* Instead of counting accidents, extract the number of injuries and perform the same analysis.

In [ ]:
!sudo pip install -U -q PyMySQL sqlalchemy sql_magic 

In [ ]:
from sqlalchemy import create_engine

conn_string = 'mysql+pymysql://{user}:{password}@{host}/{db}?charset=utf8mb4'.format(
    host = 'db.ipeirotis.org', 
    user = 'student',
    password = 'dwdstudent2015', 
    db = 'collisions',
    encoding = 'utf8mb4')

mysql_conn = create_engine(conn_string).connect()

sql = '''
  SELECT date_format(DATE_TIME,'%%Y-%%m-%%d') AS acc_date, COUNT(*) AS accidents 
  FROM collisions.collisions 
  GROUP BY date_format(DATE_TIME,'%%Y-%%m-%%d')
  ORDER BY date_format(DATE_TIME,'%%Y-%%m-%%d')
'''

acc = pd.read_sql(sql, con=mysql_conn)
acc['acc_date'] = pd.to_datetime(acc['acc_date'])
acc = acc.set_index('acc_date').resample('1D').sum()

In [ ]:
acc.plot()

### Solution

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# We decompose assumming a 12-month periodicity. 
# We can also specify a multiplicative instead of an additive model
# The additive model is Y[t] = T[t] + S[t] + e[t]
# The multiplicative model is Y[t] = T[t] * S[t] * e[t]

period = 365 # We have daily observations, and we consider one year (365 days) 
             # as the maximum seasonality period
decomposition = seasonal_decompose(acc['accidents'], 
                                   model='mutiplicative', 
                                   freq=period, 
                                   extrapolate_trend='freq'
                )  
fig = decomposition.plot()

In [ ]:
# Plot the trend
decomposition.trend.plot()

In [ ]:
# Plot the seasonal component for the first year of data
decomposition.seasonal[0:period].plot()

## Advanced: Time Series Window operations: Rolling / Expanding / EW

One question that comes up when we have a periodic time series is: "How can I figure out the overall trend?". In the examples above, we relied on a "black box" where we simply asked for the time series to be decomposed into a trend, seasonal, and residual component. Now, let's dig a bit deeper on how we can extract trend components that are unaffected by seasonality.

For that, we often rely on "window" functions, that operate over a set of continuous time series points. For example, if we have a time series that has a 12-month seasonality, we can take the 12-month average, which will not exhibit seasonality, but will capture the trend. 

These windows functions are common time series operations. Pandas provides support for various types of windows. Here are a few that are commonly used: 
* [Rolling window](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rolling.html): We compute the function over a time period equal to a window
* [Expanding](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.expanding.html): We compute the function over a period of 1, 2, 3,... instances.
* [Exponential weighting](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.ewm.html): We keep a window of a fixed size but we weight less and less (exponentially) the old data points.

In [ ]:
# Use different linestyle, and use high alpha to make the series less visually prominent
df['Sales'].plot(label='Raw', linestyle ="--", alpha=0.25)

# Plot the 12-month moving average
df['Sales'].rolling(12).mean().plot(label='12M MA', alpha=0.75)

# Plot the expanding mean. This is the mean of the series from the beginning till that point in time
df['Sales'].expanding().mean().plot(label='Expanding', alpha=0.75)

# Plot the exponentially weighted moving average. This moving average weighs more heavily the newer
# data points and weighs less the old ones. 
df['Sales'].ewm(halflife=12).mean().plot(label='EWMA (halflife 12M)', alpha=0.75)

# places the legend to the right side (1) and middle of the y-axis (0.5)
plt.legend(bbox_to_anchor=(1, .5)) 
plt.tight_layout()